<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V3/Vers%C3%A3o_3_(ajustes_de_hiperpar%C3%A2metros_DT%2C_RF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este notebook apresenta um processo de otimização de hiperparâmetros para o modelo Decision tree e Random Forest, utilizando dados extraídos da base SEER, amplamente utilizada em pesquisas oncológicas. O objetivo é ajustar os parâmetros do modelo de forma iterativa, maximizando sua performance com base na métrica AUC-ROC.

### Parte 1:  Importação das Bibliotecas e Carregamento do Dataset

Nesta etapa, importamos as bibliotecas necessárias para análise e carregamos o dataset. Realizamos uma verificação inicial para identificar e remover valores faltantes e definimos as variáveis preditoras (X) e as variáveis alvo (y), preparando os dados para o pré-processamento e a modelagem.

In [ ]:
# Importação das bibliotecas essenciais para análise de dados, visualização e aprendizado de máquina
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importação de funções e classes específicas para pré-processamento, modelagem e avaliação
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix,
                             classification_report, precision_recall_curve, average_precision_score)
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/Reprodu-o/main/export.csv')

# Verificar se existem valores faltantes (NaN) e exibir quantos valores faltantes existem por coluna
print("Valores faltantes por coluna:\n", df.isnull().sum())

# Remover as linhas com valores faltantes
df.dropna(inplace=True)

# Definir as variáveis preditoras (X)
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis',
        'Origin recode NHIA (Hispanic, Non-Hisp)']]

# Definir as variáveis alvo: Metástase no fígado e no pulmão
y_liver = df['SEER Combined Mets at DX-liver (2010+)']
y_lung = df['SEER Combined Mets at DX-lung (2010+)']



Valores faltantes por coluna:
 Patient ID                                         0
Age recode with <1 year olds                       0
Sex                                                0
Race recode (White, Black, Other)                  0
Histologic Type ICD-O-3                            0
Grade Recode (thru 2017)                           0
Primary Site                                       0
Derived AJCC T, 7th ed (2010-2015)                 0
Derived AJCC N, 7th ed (2010-2015)                 0
CS tumor size (2004-2015)                          0
CEA Pretreatment Interpretation Recode (2010+)     0
Tumor Deposits Recode (2010+)                      0
Marital status at diagnosis                        0
Origin recode NHIA (Hispanic, Non-Hisp)            0
SEER Combined Mets at DX-lung (2010+)             15
SEER Combined Mets at DX-liver (2010+)            12
SEER Combined Mets at DX-bone (2010+)             14
ICD-O-3 Hist/behav                                 0
ICD-O-3 Hist/be

###Parte 2:  Preparação das Variáveis Alvo e Codificação de Variáveis Categóricas

Nesta etapa, preparamos as variáveis alvo (y), combinando as informações de metástase hepática e pulmonar em uma coluna binária para indicar a presença de metástase. Também aplicamos LabelEncoder para transformar variáveis categóricas de X em valores numéricos, facilitando o uso dos dados em modelos de aprendizado de máquina.

In [ ]:
# Concatenar as variáveis alvo 'y_liver' e 'y_lung' em um DataFrame 'y' para problemas multi-label
y = pd.concat([y_liver, y_lung], axis=1)

# Aplicar codificação a variáveis categóricas em 'X' usando LabelEncoder, para prepará-las para o modelo
for col in X.columns:
    if X[col].dtype == 'object':  # Verifica se a coluna é categórica (strings)
        X[col] = LabelEncoder().fit_transform(X[col])

# Função para combinar as informações de metástase hepática e pulmonar em uma coluna binária 'Binary Mets'
# (0 = sem metástase, 1 = com metástase)
def combine_mets_binary(row):
    if row['SEER Combined Mets at DX-liver (2010+)'] == 'Yes' or row['SEER Combined Mets at DX-lung (2010+)'] == 'Yes':
        return 1  # Com metástase
    else:
        return 0  # Sem metástase

# Aplicar a função para criar a nova coluna binária 'Binary Mets' em 'y'
y['Binary Mets'] = y.apply(combine_mets_binary, axis=1)

# Verificar se 'X' e 'y' têm o mesmo número de amostras
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Salvar o DataFrame 'y' em um arquivo CSV para referência futura ou análise adicional
y.to_csv('/content/Y.csv')

<ipython-input-2-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-2-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Tamanho de X: 53448
Tamanho de y: 53448


###3. Otimização de hiperparâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np

# Modelos ajustados
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

# Dicionário com hiperparâmetros ajustados
param_grids = {
    "Decision Tree": {
        "max_depth": [5, 10, 15, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 5, 10],
        "criterion": ["gini", "entropy"]
    },
    "Random Forest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [10, 15, 20, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 5, 10],
        "bootstrap": [True, False],
        "criterion": ["gini", "entropy"]
    }
}

# Preparar os resultados
best_params = []

# Rodar GridSearchCV para cada modelo
for model_name, model in models.items():
    print(f"\nBuscando os melhores parâmetros para: {model_name}")

    # Definir os hiperparâmetros para o modelo atual
    param_grid = param_grids[model_name]

    # Configurar validação cruzada estratificada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Instanciar o GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring="roc_auc",  # Métrica de avaliação
        cv=cv,
        n_jobs=-1,
        verbose=1
    )

    # Ajustar o GridSearch
    grid_search.fit(X, y['Binary Mets'])

    # Salvar os melhores parâmetros e a melhor pontuação
    best_params.append({
        "Modelo": model_name,
        "Melhores Parâmetros": grid_search.best_params_,
        "Melhor Pontuação (AUC-ROC)": grid_search.best_score_
    })

    print(f"Melhores parâmetros para {model_name}: {grid_search.best_params_}")
    print(f"Melhor pontuação (AUC-ROC): {grid_search.best_score_:.4f}")

# Salvar os resultados em um arquivo TXT
with open("melhores_hiperparametros.txt", "w") as file:
    file.write("Resultados da Busca de Hiperparâmetros (GridSearchCV):\n\n")
    for result in best_params:
        file.write(f"Modelo: {result['Modelo']}\n")
        file.write(f"  Melhores Parâmetros:\n")
        for param, value in result['Melhores Parâmetros'].items():
            file.write(f"    {param}: {value}\n")
        file.write(f"  Melhor Pontuação (AUC-ROC): {result['Melhor Pontuação (AUC-ROC)']:.4f}\n")
        file.write("-" * 50 + "\n")

print("Resultados salvos no arquivo 'melhores_hiperparametros.txt'.")



Buscando os melhores parâmetros para: Decision Tree
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Melhores parâmetros para Decision Tree: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2}
Melhor pontuação (AUC-ROC): 0.8590

Buscando os melhores parâmetros para: Random Forest
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Melhores parâmetros para Random Forest: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 300}
Melhor pontuação (AUC-ROC): 0.8839
Resultados salvos no arquivo 'melhores_hiperparametros.txt'.
